In [7]:
# Import statements
import pandas as pd

In [2]:
# HPI Data

In [9]:
# Demographic Data

In [10]:
# GDP Data

In [11]:
# Unemployment data

In [12]:
# Industrial production

In [13]:
# Volatility Data

In [14]:
# Inflation Data